# Movie prefs 2 vec
I want to build a taste vector for each user to recommend content.



embedding-based recommendation system



https://netflixtechblog.com/foundation-model-for-personalized-recommendation-1a0bd8e02d39


#### Summary of how netflix does it.

1. User Interactions = “Tokens”
2. Transformer-Based Architecture
3. Next-Token Prediction, But For Movies
4. Features, Not Just IDs
5. Cold Start Problem (New Content)
6. More data + bigger models = better recommendations, just like LLMs.


In [25]:
import kagglehub
import shutil
import os

import kagglehub

# Download latest version
path = kagglehub.dataset_download("grouplens/movielens-20m-dataset")

print("Path to dataset files:", path)

# Move the downloaded files to the destination folder
for filename in os.listdir(path):
    source_file = os.path.join(path, filename)
    destination_file = os.path.join("data", filename)
    
    # Move the file
    shutil.move(source_file, destination_file)

Path to dataset files: /home/morrisubuntu/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1


In [26]:
import pandas as pd

# Load the movies data into a DataFrame
movies_df = pd.read_csv("data/movies.csv")
movies_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/movies.csv'

In [ ]:
links_df = pd.read_csv("data/links.csv")
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
ratings_df = pd.read_csv("data/ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
tags_df = pd.read_csv("data/tags.csv")
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
# Find the movie in movies_df with movieId 60756
movie_60756 = movies_df[movies_df['movieId'] == 60756]

tags_60756 = tags_df[tags_df['movieId'] == 60756]

# Display movie info and user tags
print("Movie info:")
print(movie_60756)
print("\nUser tags for this movie:")
print(tags_60756[['userId', 'tag']])

Movie info:
      movieId                 title  genres
6801    60756  Step Brothers (2008)  Comedy

User tags for this movie:
     userId              tag
0         2            funny
1         2  Highly quotable
2         2     will ferrell
205      62           comedy
206      62            funny
207      62     will ferrell
909     424            funny
910     424     will ferrell


In [ ]:
print(f"Number of unique users: {ratings_df['userId'].nunique()}")
print(f"Number of unique movies: {ratings_df['movieId'].nunique()}")
print(f"Number of ratings: {len(ratings_df)}")
print(f"Number of tags: {len(tags_df)}")



Number of unique users: 610
Number of unique movies: 9724
Number of ratings: 100836
Number of tags: 3683


In [ ]:
print(f"\nRating statistics:")
print(ratings_df['rating'].describe())


Rating statistics:
count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64


In [ ]:
# Extract all unique genres from the dataset
all_genres = set()
for genres in movies_df['genres']:
    if genres != '(no genres listed)':
        all_genres.update(genres.split('|'))

print(f"Number of unique genres: {len(all_genres)}")
print(f"Unique genres: {sorted(all_genres)}")

Number of unique genres: 19
Unique genres: ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [ ]:
reviews_per_user = ratings_df.groupby('userId').size()
print(reviews_per_user.head())
print(f"\nAverage number of reviews per user: {reviews_per_user.mean():.2f}")

userId
1    232
2     29
3     39
4    216
5     44
dtype: int64

Average number of reviews per user: 165.30


In [ ]:
# For each movie, split the genres string into a list (or empty list if no genres)
def split_genres(genres):
    if genres == '(no genres listed)':
        return []
    else:
        return genres.split('|')

genre_list = movies_df['genres'].apply(split_genres) 


In [ ]:
# Create genre indicators for each movie
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer # because we can onehot encode this one 


mlb = MultiLabelBinarizer(classes=sorted(all_genres))
genre_indicators = mlb.fit_transform(genre_list)
genre_df = pd.DataFrame(genre_indicators, columns=mlb.classes_)
genre_df['movieId'] = movies_df['movieId']

genre_df.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieId
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,4
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5


In [ ]:
# Method 1: User-Genre preference vectors
# Merge ratings with genres
user_ratings = ratings_df.merge(movies_df[['movieId', 'genres']], on='movieId')
user_ratings

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Horror
100834,610,168252,5.0,1493846352,Action|Sci-Fi


## NOTES:
- we have a high number of reviews per user.
- many movies
- low amount of users
- im thinking about splitting the users into training and test data, For this i mabye LOO is the best option because we have a low amount of users to test on.


"Leave One Out" (LOO) is a term most commonly used in statistics and machine learning, referring to a specific type of cross-validation technique called Leave-One-Out Cross-Validation (LOOCV).

In [ ]:
# Create a mapping of genres to index positions
genre_list = sorted(all_genres)
genre_to_idx = {genre: idx for idx, genre in enumerate(genre_list)}

print(genre_to_idx)

user_taste_vectors = {} # init taste vector

{'Action': 0, 'Adventure': 1, 'Animation': 2, 'Children': 3, 'Comedy': 4, 'Crime': 5, 'Documentary': 6, 'Drama': 7, 'Fantasy': 8, 'Film-Noir': 9, 'Horror': 10, 'IMAX': 11, 'Musical': 12, 'Mystery': 13, 'Romance': 14, 'Sci-Fi': 15, 'Thriller': 16, 'War': 17, 'Western': 18}


In [ ]:

def create_user_taste_vector(user_id):
    user_data = user_ratings[user_ratings['userId'] == user_id]
    if len(user_data) == 0:
        return None
    genre_vector = np.zeros(len(genre_list))
    rating_sum = user_data['rating'].sum()
    for _, row in user_data.iterrows():
        if row['genres'] != '(no genres listed)':
            for genre in row['genres'].split('|'):
                genre_index = genre_to_idx[genre]
                # Weight by rating (normalized)
                genre_vector[genre_index] += row['rating'] / rating_sum
    
    return genre_vector

sample_users = ratings_df['userId'].unique()[:10]  # Just take first 10 users for demonstration

for user_id in sample_users:
    user_taste_vectors[user_id] = create_user_taste_vector(user_id)

sample_user = sample_users[0]
taste_vector = user_taste_vectors[sample_user]

user_taste_df = pd.DataFrame({
    'Genre': genre_list,
    'Preference': taste_vector
})

user_taste_df = user_taste_df.sort_values('Preference', ascending=False)

print(f"User {sample_user}'s top genre preferences:")
print(user_taste_df.head(10))

User 1's top genre preferences:
        Genre  Preference
0      Action    0.384008
1   Adventure    0.368213
4      Comedy    0.350444
7       Drama    0.304047
16   Thriller    0.225074
8     Fantasy    0.199408
5       Crime    0.193485
3    Children    0.188549
15     Sci-Fi    0.166831
2   Animation    0.134255


## Splitting Data into Training and Test Sets
We'll split the ratings into training and test sets for evaluating our recommendation vector.

In [ ]:
# Create a user-movie rating matrix (users as rows, movies as columns)
user_movie_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating')

# Split data into training (80%) and test (20%) sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)
print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

# Create user-movie matrix from training data only
user_movie_matrix = train_df.pivot_table(index='userId', columns='movieId', values='rating')

Training set size: 80668
Test set size: 20168


## Latent-Factor Taste Profiles
Use SVD on the user–movie matrix to create low-dimensional taste vectors based on other users’ ratings.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Prepare matrix
matrix = user_movie_matrix.fillna(0)
# Fit SVD
n_components = 20
svd = TruncatedSVD(n_components=n_components, random_state=42)
user_factors = svd.fit_transform(matrix)
movie_factors = svd.components_.T

# Build DataFrames
latent_cols = [f'latent_{i}' for i in range(n_components)]
user_profile_df = pd.DataFrame(user_factors, index=matrix.index, columns=latent_cols)
movie_profile_df = pd.DataFrame(movie_factors, index=matrix.columns, columns=latent_cols)

# Display a sample user's taste vector
print(f"User {sample_user} latent taste vector:")
print(user_profile_df.loc[sample_user])

User 1 latent taste vector:
latent_0     24.895775
latent_1    -11.545541
latent_2      5.430155
latent_3      1.006397
latent_4     -9.879566
latent_5     -5.294737
latent_6      1.782714
latent_7      2.688068
latent_8      1.262096
latent_9      1.067685
latent_10     2.609840
latent_11     0.932714
latent_12     1.619247
latent_13     6.741034
latent_14    -2.782239
latent_15     0.485790
latent_16     5.798586
latent_17     0.078430
latent_18    -5.610701
latent_19    -0.157358
Name: 1, dtype: float64


### Finding Similar Movies
Given a movie ID, find its nearest neighbors in latent space.

In [ ]:
def similar_movies(movie_id, top_n=10):
    if movie_id not in movie_profile_df.index:
        return None
    vec = movie_profile_df.loc[movie_id].values.reshape(1, -1)
    sims = cosine_similarity(vec, movie_profile_df.values)[0]
    sim_series = pd.Series(sims, index=movie_profile_df.index)
    top = sim_series.drop(index=movie_id).nlargest(top_n)
    return movies_df[movies_df.movieId.isin(top.index)][['movieId', 'title']].assign(similarity=top.values)

# Example: neighbors for movie 3
neighbors = similar_movies(3)
print(neighbors)

      movieId                   title  similarity
377       433      Clean Slate (1994)    0.819718
449       514         Ref, The (1994)    0.796079
475       542       Son in Law (1993)    0.780327
550       656            Eddie (1996)    0.776808
619       784   Cable Guy, The (1996)    0.775038
643       829  Joe's Apartment (1996)    0.763502
1129     1476    Private Parts (1997)    0.736000
1866     2478   ¡Three Amigos! (1986)    0.735588
2534     3394       Blind Date (1987)    0.735552
3201     4317   Love Potion #9 (1992)    0.732638


## User Taste Profiles in Latent Space
Construct each user’s taste vector by combining movie latent factors weighted by their ratings, then recommend based on this profile.

In [ ]:
def build_user_latent_vector(user_id):
    user_r = train_df[train_df.userId == user_id][['movieId','rating']]
    if user_r.empty: return None
    # Normalize ratings
    norm = user_r['rating'] / user_r['rating'].sum()
    # Get corresponding movie latent vectors
    vecs = movie_profile_df.loc[user_r['movieId']]
    # Weight and sum
    weighted = vecs.mul(norm.values, axis=0)
    return weighted.sum()

def recommend_user_profile(user_id, top_n=10):
    uvec = build_user_latent_vector(user_id).values.reshape(1, -1)
    sims = cosine_similarity(uvec, movie_profile_df.values)[0]
    sim_series = pd.Series(sims, index=movie_profile_df.index)
    seen = train_df[train_df.userId == user_id].movieId
    recs = sim_series.drop(index=seen, errors='ignore').nlargest(top_n)
    # Get recommendations DataFrame
    rec_df = movies_df[movies_df.movieId.isin(recs.index)][['movieId', 'title', 'genres']].assign(similarity=recs.values)
    return rec_df

# Example: recommendations for sample user
user_recs = recommend_user_profile(sample_user)
print(f"Recommendations via latent profile for user {sample_user}:")
print(user_recs)

Recommendations via latent profile for user 1:
      movieId                                        title  \
371       427                         Boxing Helena (1993)   
793      1036                              Die Hard (1988)   
902      1200                                Aliens (1986)   
1071     1391                         Mars Attacks! (1996)   
1183     1580             Men in Black (a.k.a. MIB) (1997)   
1576     2115  Indiana Jones and the Temple of Doom (1984)   
1726     2320                             Apt Pupil (1998)   
2195     2918              Ferris Bueller's Day Off (1986)   
2652     3550                            The Hunger (1983)   
3390     4613                                   K-9 (1989)   

                              genres  similarity  
371   Drama|Mystery|Romance|Thriller    0.675404  
793            Action|Crime|Thriller    0.661320  
902   Action|Adventure|Horror|Sci-Fi    0.619086  
1071            Action|Comedy|Sci-Fi    0.610405  
1183           

In [ ]:
user1_ratings = train_df[train_df['userId'] == 1].merge(
    movies_df[['movieId', 'title', 'genres']], on='movieId'
).sort_values('rating', ascending=False)
print(user1_ratings[['title', 'rating', 'genres']].head(10))

                                    title  rating  \
176                     Fight Club (1999)     5.0   
177                Charlotte's Web (1973)     5.0   
180               Crocodile Dundee (1986)     5.0   
181              American Tail, An (1986)     5.0   
8                       Spaceballs (1987)     5.0   
9                           Dr. No (1962)     5.0   
12          NeverEnding Story, The (1984)     5.0   
13                           Shaft (1971)     5.0   
14             Black Cauldron, The (1985)     5.0   
17   Road Warrior, The (Mad Max 2) (1981)     5.0   

                                   genres  
176           Action|Crime|Drama|Thriller  
177                    Animation|Children  
180                      Adventure|Comedy  
181   Adventure|Animation|Children|Comedy  
8                           Comedy|Sci-Fi  
9               Action|Adventure|Thriller  
12             Adventure|Children|Fantasy  
13            Action|Crime|Drama|Thriller  
14   Adventure|Anima